In [ ]:
import _init_paths
from fast_rcnn.config import cfg
from fast_rcnn.test import im_detect
from fast_rcnn.nms_wrapper import nms
from utils.timer import Timer
import matplotlib.pyplot as plt
import numpy as np
import scipy.io as sio
import caffe, os, sys, cv2
import argparse


In [ ]:
CLASSES = ('__background__','Face')

NETS = {'vgg16': ('VGG16',
          '/home/saadi/caffe-face-frcnn/models/vgg16_faster_rcnn_iter_80000.caffemodel')}

cfg.TEST.HAS_RPN = True  # Use RPN for proposals

prototxt = '/home/saadi/caffe-face-frcnn/models/face/VGG16/faster_rcnn_end2end/test.prototxt'
caffemodel = '/home/saadi/caffe-face-frcnn/models/vgg16_faster_rcnn_iter_80000.caffemodel'
net = caffe.Net(prototxt, caffemodel, caffe.TEST)
print '\n\nLoaded network {:s}'.format(caffemodel)

In [ ]:
### creates  a list of paths of images
import os
yield_images= [os.path.join('/home/saadi/Desktop/test_xml _write/images', file) for file in os.listdir('/home/saadi/Desktop/test_xml _write/images') if file.endswith('.jpg')]
print(yield_images)
yield_images.sort(key=lambda f: int(''.join(filter(str.isdigit, f))))# reorganizes the list
print('###################################################')
print(yield_images)

In [ ]:
### Creates empty text files 
import os
for f in os.listdir('/home/saadi/Desktop/test_xml _write/images'):
    if f.endswith('.jpg'):
        file_path = os.path.join('/home/saadi/Desktop/test_xml _write/textfile_faceloactions',f.replace('.jpg', '.txt'))
        with open(file_path, "w+"):
            pass

In [ ]:
### creates a list of paths of text files to save face locations 
import os
txt_list= [os.path.join('/home/saadi/Desktop/test_xml _write/textfile_faceloactions', file) for file in os.listdir('/home/saadi/Desktop/test_xml _write/textfile_faceloactions') if file.endswith('.txt')]
print(txt_list)
txt_list.sort(key=lambda f: int(''.join(filter(str.isdigit, f))))# reorganizes the list
print('###################################################')
print(txt_list)

In [ ]:
"""Detect object classes in an image using pre-computed object proposals."""
#yield_images=['/home/saadi/Desktop/test_xml _write/images/0.jpg']
# Load the demo image
x=0

No_of_faces=0
for i in yield_images:
    im_file = i
    print(im_file)
    im = cv2.imread(im_file)
    
    img = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
 
    img_h, img_w, _ = np.shape(im)
    # Detect all object classes and regress object bounds
    timer = Timer()
    timer.tic()
    scores, boxes = im_detect(net, im)
    timer.toc()
    print ('Detection took {:.3f}s for '
          '{:d} object proposals').format(timer.total_time, boxes.shape[0])
    #print ('The boxses are =',boxes)
   # print ('The scores are =', scores)
    #print('The type of boxses are',type(boxes))
   
    # Visualize detections for each class
    CONF_THRESH = 0.98
    NMS_THRESH = 0.3
   
    for cls_ind, cls in enumerate(CLASSES[1:]):
        cls_ind += 1 # because we skipped background
        cls_boxes = boxes[:, 4*cls_ind:4*(cls_ind + 1)]
        cls_scores = scores[:, cls_ind]
        #print(cls_boxes)
        #print('The type of cls_boxes',type(cls_boxes))
        dets = np.hstack((cls_boxes,
                          cls_scores[:, np.newaxis])).astype(np.float32)
        
        
        keep = nms(dets, NMS_THRESH)
        dets = dets[keep, :]
        #print(dets)
        #print('The type of dets is',type(dets))
        
        #vis_detections(im, cls, dets, thresh=CONF_THRESH)

        """Draw detected bounding boxes."""
       
        thresh=CONF_THRESH
        class_name= cls
        inds = np.where(dets[:, -1] >= thresh)[0]
        
        print('Number of Faces detected',len(inds))
        No_of_faces=No_of_faces + len(inds)
        
        #print(type(inds))
        #print(inds)
        
        r=len(inds) ## to save number of faces in text file
        
        if len(inds) != 0:
            im = im[:, :, (2, 1, 0)]
            fig, ax = plt.subplots(figsize=(12, 12))
            ax.imshow(im, aspect='equal')
            
            face_list=[]  ### empty list to save tuyples
            ## uncomment below to add detected number of faces to text file
            face_img_not_detected = []
            face_list.append(r)  ### use append to add integers
            face_list.extend('\n') ### next line
            
            for i in inds:
                bbox = dets[i, :4]
                #print('the bbox is dets[i, ;4]', bbox)
                score = dets[i, -1]
                
                x1 = int(bbox[0]) 
                y1 = int(bbox[1])
                x2 = int(bbox[2]) 
                y2 = int(bbox[3]) 
                
                
                
                
                
               
                w = bbox[2] - bbox[0]  # width
                h = bbox[3] - bbox[1]  #height
    
                xw1 = max(int(bbox[0] - 0.4 * w), 0)
                yw1 = max(int(bbox[1] - 0.4 * h), 0)
                xw2 = min(int(bbox[2] + 0.4 * w), img_w - 1)
                yw2 = min(int(bbox[3] + 0.4 * h), img_h - 1)
                
                
#########################       adding script to save face locations in a text file          
                
              #  lines = xw1 ### original was x, y, etc etc without brackets
             #   face_list.append(lines)
             #   face_list.extend('\n')
              #  lines = yw1
             #   face_list.append(lines)
             #   face_list.extend('\n')
             #   lines = xw2
             #   face_list.append(lines)
             #   face_list.extend('\n')
             #   lines = yw2
             #   face_list.append(lines)
             #   face_list.extend('\n')
                #print(type(lines))  ## line is a tyuple
             #   face_list.extend(lines)## make a list of all the tuyples coming in
             #   face_list.extend('\n')
                
################################                
    #### Trying to write as dlib format
    
                #lines = xw1, yw1, xw2, yw2 ### original was x, y, etc etc without brackets
                lines = x1, y1, x2, y2 #just to save close faces
                #print(type(lines))         
                face_list.extend(lines)
                face_list.extend('\n')
                 
                
    
    ###############################################3
               # cv2.rectangle(img, (xw1, yw1), (xw2, yw2), (255, 0, 0), 4)
                cv2.rectangle(img, (x1, y1), (x2, y2), (255, 0, 0), 4) # close faces
               # ax.add_patch(
               #     plt.Rectangle((bbox[0], bbox[1]),bbox[2] - bbox[0],bbox[3] - bbox[1], fill=False,
               #                   edgecolor='red', linewidth=3.5)
               #   plt.Rectangle((xw1, yw1),xw2 - xw1,yw2 - yw1, fill=False,
               #                  edgecolor='red', linewidth=3.5)
                #    )
                ax.text(bbox[0], bbox[1] - 2,
                      '{:s} {:.3f}'.format(class_name, score),
                       bbox=dict(facecolor='blue', alpha=0.5),
                       fontsize=14, color='white')

            ax.set_title(('{} detections with '
                         'p({} | box) >= {:.1f}').format(class_name, class_name,
                                                        thresh),
                        fontsize=14)
            ################################## script to save in text file
            print('The face list =',face_list)
            print(type(face_list))
            thefile = open(txt_list[x], 'w')
            print('the VAlue of X =',x)
            for item in face_list:
                if item != '\n':
                  thefile.write("%d " % item)
                else :
                  thefile.write("%s" % item)
    x=x+1
            
            
            
        ######################
    if len(inds) == 0:
            face_not_detected = i #just to save close faces
            #print(type(lines))         
            face_img_not_detected.extend(face_not_detected)
            face_img_not_detected.extend('\n')
            
            
    plt.axis('off')
    plt.tight_layout()
    plt.imshow(img)
    plt.draw()
    plt.show()
    print("the no of faces",No_of_faces)
    
print("the no of faces",No_of_faces)
file_txt = open('/home/saadi/Desktop/test_xml _write/images not detected by algorithm.txt', 'w')    
for item in face_not_detected:
    file_txt.write("%s" % item)

In [ ]:
file_txt = open('/home/saadi/Desktop/test_xml _write/images not detected by algorithm.txt', 'w') 
print(face_not_detected)
for item in face_not_detected:
    file_txt.write("%s" % item)